In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal as sig
from scipy.fftpack import fft
from math import pi

In [ ]:
T = 600
Sample_rate= 100
SegmentDuration = 100
N = T*Sample_rate
t = np.arange(N)/Sample_rate
N0 = 0.1
freq = 2.7
# Original Signal
c=np.sin(2*pi*freq*t)

#Noise
noise = np.random.normal(0,N0,N) 

#Main chanel
x = 3.0*c+noise

#Witness chanel
y = 5.0*c+noise
plt.figure(figsize=(15,4))
plt.plot(t[0:Sample_rate],x[0:Sample_rate] , label=r' main chanel')
plt.plot(t[0:Sample_rate],y[0:Sample_rate] , label=r' witness chanel')
plt.legend()
plt.grid()

In [ ]:
#Amplitude spectrum
f1 , psdx = sig.welch(x,Sample_rate,'flattop',1024,scaling='spectrum')
f2 , psdy = sig.welch(y,Sample_rate,'flattop',1024,scaling='spectrum')
plt.figure(figsize=(15,4))
plt.semilogy(f1,psdx, label=r' main chanel')
plt.semilogy(f1,psdy, label=r' witness chanel')
plt.xlabel('Frequency')
plt.ylabel('PSD')
plt.grid()
plt.legend()

In [ ]:
#Wiener filter
Seg_Wiener = 10
x_wiener = x[0:SegmentDuration*Sample_rate*Seg_Wiener]
y_wiener = y[0:SegmentDuration*Sample_rate*Seg_Wiener]

for i in range(Seg_Wiener):
    tempx = x_wiener[(i*SegmentDuration*Sample_rate):(i*SegmentDuration*Sample_rate)]
    tempy = y_wiener[(i*SegmentDuration*Sample_rate):(i*SegmentDuration*Sample_rate)]
    if i==0:
        T_f = np.fft.irfft(sig.hann(len(tempx))*tempx)/np.fft.rfft(sig.hann(len(tempy))*tempy)
    else:
        T_f += np.fft.irfft(sig.hann(len(tempx))*tempx)/np.fft.rfft(sig.hann(len(tempy))*tempy)
T_f = T_f/Seg_Wiener

freq = np.fft.rfftfreq(SegmentDuration*Sample_rate,d=1./Sample_rate)

#Coherence Cut
f ,Cxy = sig.cohernce(x_wiener,y_wiener,fs=Sample_rate,window='hann',nperseg=SegmentDuration*Sample_rate,noverlap=None,nfft=None,deternd='constant',axis=-1)
T_f[Cxy > CThreshold] = 0

#Plot Filter
plt.figure(figsize=(15,4))

plt.subplot(121)
plt.plot(freq, np.abs(T_f))
plt.grid()
plt.xlabel('Freq(Hz)')
plt.ylabel('Amplitude')

plt.subplot(122)
plt.plot(freq, np.angle(T_f,deg=True))
plt.grid()
plt.ylim(-180,180)
plt.xlabel('Freq(Hz)')
plt.ylabel('Phase (degrees)')
plt.tight_layout()

In [ ]:
#Wiener Subtraction
tmp = np.fft.irfft(T_f)
tmp_len = len(tmp)
tmp = sig.hann(tmp_len)*np.roll(tmp, np.int(tmp_len/2))
tmp.resize(len(witness)) # pad with zeros to lenght of witness
f_long = np.fft.rfft(np.roll(tmp,-np.int(tmp_len/2)))
freq1 = np.fft.rfftfreq(len(y),d=1./Sample_rate)
tmp = np.fft.rfft(y)*T_f_long
est_subtract = np.fft.irfft(tmp)

#Plot
plt.figure(figsize=(15,4))

plt.subplot(121)
plt.plot(t,x,label='original')
plt.plot(t,est_subtract,label='subtraction signal')
plt.plot(t,x-est_subtract,label='residual')
plt.grid()
plt.legend()
plt.xlabel('Time (sec)')
plt.ylabel('Amplitude')

plt.subplot(122)
plt.plot(t,x,label='original')
plt.plot(t,est_subtract,label='subtraction signal')
plt.plot(t,x-est_subtract,label='residual')
plt.grid()
plt.xlabel('Amplitude')
plt.ylabel('Time (sec)')
plt.tight_layout()
plt.legend()